# QSVM

In [13]:
import pandas as pd
from qiskit import Aer, IBMQ
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel, FidelityQuantumKernel
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('datasets/custom_dataset.csv', index_col=0)
df.head(3)

,feature_1,feature_2,label
0,3.109987,2.890013,0.0
1,3.038349,2.961651,0.0
2,3.133267,2.866733,0.0


In [3]:
feature_dimension = df.shape[1] - 1
print(f"Feature dimension: {feature_dimension}")

Feature dimension: 2


In [4]:
# Split the data into training and test sets
all_features = df.copy()
all_labels = all_features.pop('label')

train = df.sample(frac=0.75, random_state=42)
test = df.drop(train.index)

# Separate the features from the labels
train_features = train.copy()
test_features = test.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

In [24]:
algorithm_globals.random_seed = 42
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='zhaw-1')
backend = provider.get_backend('ibmq_belem')
# backend = Aer.get_backend("aer_simulator")
quantum_instance = QuantumInstance(backend, shots=1024)

In [25]:
feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear')
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
# kernel = FidelityQuantumKernel(fidelity=ComputeUncompute(sampler=Sampler()))
svc = SVC(kernel=kernel.evaluate)

In [26]:
svc.fit(train_features, train_labels)
score = svc.score(test_features, test_labels)
print(f"QSVM score: {score}")

QSVM score: 0.72


In [27]:
print(svc.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': <bound method QuantumKernel.evaluate of <qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x293523880>>, 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [28]:
scores = cross_val_score(svc, all_features, all_labels, cv=10)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

KeyboardInterrupt: 

In [9]:
classical_svc = SVC()
classical_svc.fit(train_features, train_labels)
score = classical_svc.score(test_features, test_labels)

In [10]:
print(f"Classical SVC score: {score}")

Classical SVC score: 0.72


In [11]:
scores = cross_val_score(classical_svc, all_features, all_labels, cv=10)
scores

array([0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.6])

In [12]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.69 accuracy with a standard deviation of 0.03
